# Calcualte MRR and MAP

In [ ]:
import sys
import statistics

from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd

# Descides, which k 
MaxMRRRank = 10




def load_reference(path_to_reference):
    with open(path_to_reference, 'r') as f:
        qids_to_relevant_passageids = load_reference_from_stream(f)
    return qids_to_relevant_passageids


def load_reference_from_stream(f):
    qids_to_relevant_passageids = {}
    for l in f:
        try:
            l = l.strip().split('\t')
            qid = int(l[0])
            if qid in qids_to_relevant_passageids:
                pass
            else:
                qids_to_relevant_passageids[qid] = []
            qids_to_relevant_passageids[qid].append(int(l[1]))
        except:
            raise IOError('\"%s\" is not valid format' % l)
    return qids_to_relevant_passageids


def quality_checks_qids(qids_to_relevant_passageids, qids_to_ranked_candidate_passages):
    message = ''
    allowed = True

    # Create sets of the QIDs for the submitted and reference queries
    candidate_set = set(qids_to_ranked_candidate_passages.keys())
    ref_set = set(qids_to_relevant_passageids.keys())

    # Check that we do not have multiple passages per query
    for qid in qids_to_ranked_candidate_passages:
        # Remove all zeros from the candidates
        duplicate_pids = set(
            [item for item, count in Counter(qids_to_ranked_candidate_passages[qid]).items() if count > 1])

        if len(duplicate_pids - set([0])) > 0:
            message = "Cannot rank a passage multiple times for a single query. QID={qid}, PID={pid}".format(
                qid=qid, pid=list(duplicate_pids)[0])
            allowed = False

    return allowed, message


def load_candidate(path_to_candidate):
    with open(path_to_candidate, 'r') as f:
        qid_to_ranked_candidate_passages = load_candidate_from_stream(f)
    return qid_to_ranked_candidate_passages

def load_candidate_from_stream(f):
    qid_to_ranked_candidate_passages = {}
    for l in f:
        try:
            l = l.strip().split('\t')
            int(l[0].split('.')[0])
            qid = int(l[0].split('.')[0])
            pid = int(l[1].split('.')[0])
            rank = int(l[2].split('.')[0])
            if qid in qid_to_ranked_candidate_passages:
                pass
            else:
                # By default, all PIDs in the list of 1000 are 0. Only override those that are given
                tmp = [0] * 1000
                qid_to_ranked_candidate_passages[qid] = tmp
            qid_to_ranked_candidate_passages[qid][rank - 1] = pid
        except:
            raise IOError('\"%s\" is not valid format' % l)
    return qid_to_ranked_candidate_passages



def compute_metrics(qids_to_relevant_passageids, qids_to_ranked_candidate_passages):
    all_scores = {}
    MRR = 0
    MAP = 0
    ranking = []
    count = 0
    # Removes the queries, for which no relevant passage is in the top1000 
    print("Before", len(qids_to_ranked_candidate_passages))
    #qids_to_ranked_candidate_passages = remove_missing_right_values(qids_to_relevant_passageids, qids_to_ranked_candidate_passages)
    print("After", len(qids_to_ranked_candidate_passages))
    for qid in qids_to_ranked_candidate_passages:
        if qid in qids_to_relevant_passageids:
            count = count + 1
            ranking.append(0)
            target_pid = qids_to_relevant_passageids[qid]
            candidate_pid = qids_to_ranked_candidate_passages[qid]
            target_pid = set(target_pid).intersection(candidate_pid)
            if len(target_pid) > 1:
              print('length ', qid, len(target_pid))
            count_t_pid = 0
            # computes MRR
            for i in range(0, MaxMRRRank):
                if candidate_pid[i] in target_pid:
                    MRR += 1 / (i + 1)
                    ranking.pop()
                    ranking.append(i + 1)
                    break
            MAP_here = 0
            # computes MAP
            for i in range(0, 1000):
                if candidate_pid[i] in target_pid:
                    count_t_pid += 1
                    MAP_here += count_t_pid / (1+i)
                    #print('Map_here' , MAP_here)
            MAP += MAP_here / len(target_pid)
            #print("QID, Rankings",qid,ranking)
    print("count: ",count)
    if len(ranking) == 0:
        raise IOError("No matching QIDs found. Are you sure you are scoring the evaluation set?")

    MRR = MRR * (1 / len(qids_to_ranked_candidate_passages))
    MAP = MAP * (1 / len(qids_to_ranked_candidate_passages))
    all_scores['MRR @10'] = MRR
    all_scores['MAP'] = MAP
    all_scores['QueriesRanked'] = len(qids_to_ranked_candidate_passages)
    return all_scores

# Removes the queries, for which no relevant passage is in the top1000 
def remove_missing_right_values(qids_to_relevant_passageids, qids_to_ranked_candidate_passages):
    not_possible = []
    for qid in qids_to_ranked_candidate_passages:
        target_pid = qids_to_relevant_passageids[qid]
        if len(list(set(target_pid) & set(qids_to_ranked_candidate_passages[qid]))) < 1:
            not_possible.append(qid)
    for ids in not_possible:
        del qids_to_ranked_candidate_passages[ids]
    return qids_to_ranked_candidate_passages

def compute_metrics_from_files(path_to_reference, path_to_candidate, perform_checks=True):
    qids_to_relevant_passageids = load_reference(path_to_reference)
    qids_to_ranked_candidate_passages = load_candidate(path_to_candidate)
    print(len(qids_to_ranked_candidate_passages))
    if perform_checks:
        allowed, message = quality_checks_qids(qids_to_relevant_passageids, qids_to_ranked_candidate_passages)
        if message != '': print(message)
    return compute_metrics(qids_to_relevant_passageids, qids_to_ranked_candidate_passages)


def main():
    # Ground truth
    path_to_reference = "/content/drive/MyDrive/Eva/qrels.dev.tsv"

    # Our rankings
    path_to_candidate = "/content/drive/MyDrive/Eva/final_result_CC/all.csv"
    #path_to_candidate = "/content/drive/MyDrive/Eva/final_logreg_pointwise/all.csv"
    #path_to_candidate = "/content/drive/MyDrive/Eva/final_result_XLnet_62000/all.csv"
    #path_to_candidate = "/content/drive/MyDrive/Eva/final_result_XLnet_22000/all.csv"
    #path_to_candidate = "/content/drive/MyDrive/Eva/final_result_XLnet_142000/all.csv"
    #path_to_candidate = "/content/drive/MyDrive/Eva/Hybrid/avg.csv"
    #path_to_candidate = "/content/drive/MyDrive/Eva/Hybrid/65-25-10.csv"
    #path_to_candidate = "/content/drive/MyDrive/Eva/Hybrid/Pair_70-15-15.csv"
    #path_to_candidate = "/content/drive/MyDrive/Eva/Hybrid/45-30-25.csv"  
    #path_to_candidate = "/content/drive/MyDrive/Eva/final_result_pair_CC/all.csv"
    #path_to_candidate = "/content/drive/MyDrive/Eva/final_logreg_pairwise_binary/all.csv"
    #path_to_candidate = "/content/drive/MyDrive/Eva/final_pairwise_sum/all.csv"
    metrics = compute_metrics_from_files(path_to_reference, path_to_candidate)
    print('---------------------')
    for metric in sorted(metrics):
        print('{}: {}'.format(metric, metrics[metric]))
    print('---------------------')


if __name__ == '__main__':
    main()


# Creates the right format to use

In [ ]:
import csv

import numpy as np
from pathlib import Path
import os, glob
import pandas as pd

def ranking(Moritz = None):
    pathlist = Path("/content/drive/MyDrive/Eva/XLNet142000").rglob('*.csv')
    for path in pathlist:
        result = pd.read_csv(
            str(path),
            sep=",", header=Moritz)
        result = result.to_numpy()
        # for XLNet
        #result = result[np.argsort(result[:, 2])]
        for i, element in enumerate(result):
            result[i, 2] = i + 1
            result[i, 0] = int(result[i, 0])
            result[i, 1] = int(result[i, 1])
            result[i, 2] = int(result[i, 2])
        with open("/content/drive/MyDrive/Eva/final_result_XLnet_142000/" + str(path).split('/')[-1], 'w', newline='') as file:
            mywriter = csv.writer(file, delimiter='\t')
            mywriter.writerows(result)

def combining():
    path = r'/content/drive/MyDrive/Eva/final_result_XLnet_142000'  # use your path
    all_files = glob.glob(path + "/*.csv")

    li = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=None, sep="\t")
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    frame = frame.to_numpy()
    with open("/content/drive/MyDrive/Eva/final_result_XLnet_142000/all.csv", 'w', newline='') as file:
        mywriter = csv.writer(file, delimiter='\t')
        mywriter.writerows(frame)
ranking()
#ranking(Moritz=0)
combining()